<a href="https://www.kaggle.com/code/dascient/uacp-defining-powellscore-veracity-variables?scriptVersionId=144252576" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 2.0 UACP - Defining PowellScore & Veracity Variables
## [1.0 UAP Analytic Centralization Program](https://www.kaggle.com/code/dascient/uacp-uap-analytic-centralization-program)
<br>

## [NLP - Sentiment Intensity Analyzer](https://github.com/cjhutto/vaderSentiment) Against Reporting Comments
<br>

### In collaboration with The Scientific Coalition for UAP Studies [(SCU)](ExploreSCU.org).
Here we isolate only pertinent variables from the original dataset. We've also decided to leave open most of the code cells below; enabling transparency on foundation of all variables. 

In [ ]:
%%time
# for the sake of expeditious analysis
!pip install xlrd
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from geopandas import GeoDataFrame
import matplotlib.colors as colors
import seaborn as sns
import random as r

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        #print('Files loaded.')
        
pd.set_option('display.max_colwidth', None)

# loading first nuforc dataframe
og_df1 = pd.read_csv('/kaggle/input/ufo-sightings/ufos.csv',header=0)
df = og_df1.dropna().copy()
og_df2 = pd.read_csv('/kaggle/input/d/NUFORC/ufo-sightings/scrubbed.csv',header=0)
df2 = og_df2.dropna().copy()

#############################################

lex = pd.read_excel('/kaggle/input/scu-nlp-uap-lexicon/UFO lexicon rev2.xls',sheet_name='Sheet1',header=7)
lex = lex.dropna(how='all').drop(columns='Unnamed: 0').copy()

#############################################
# sanitize
# drop some columns, for now
df = df.drop(columns=['datetime','duration (hours/min)'])

# date posted deemed to be easily conveible to timestamp values, so i'm gonna work with that for now.
df['date posted'] = df['date posted'].astype('datetime64[ns]')


# length of comments
df['comment_length'] = [len(str(v[0:500])) for i,v in df.comments.items()]


# convert seconds to minutes
df["duration (minutes)"] = [int(v)/60 for i,v in df["duration (seconds)"].items()]


# creating Geo Point column for sopecial use below
df['Geo Point'] = df.apply(lambda x:'%s, %s' % (x['latitude'],x['longitude']),axis=1)


# let's create subsets of our 80,000 here: 
# we can implement conditionals, remove/analyze outliers, 
# & will enable for back referencing when starting to run 
# robust AI-ML modeling that would otherwise take much longer to run.

# let's create subsets from the main dataframe/reporting-data w/ respect to duration of observations
df_under100 = df[df["duration (minutes)"]<100]
df_under60 = df[df["duration (minutes)"]<60]

# random binary column for future AI-ML modeling.
a=['balloon','spacejunk','sensor_malfunction','undentified','anomalous']     
df['verified'] = pd.Series(r.choices(a,k=len(df),weights=(50, 40, 30, 20, 10)),index=df.index)

# shape-focused
circles = df[df['shape'] == 'circle']
spheres = df[df['shape'] == 'sphere']
lights = df[df['shape'] == 'light']
teardrops = df[df['shape'] == 'teardrop']

# year-month
df['year_month'] = df['date posted'].dt.to_period('M')

clear_output()
# show
print("\nOriginal dataset.")
print(f"\nReports: {len(df)} non-null dataframe.")
print("\nMatrix:",df.shape[0],"rows,",df.shape[1],"columns")
df = df.sort_values('date posted',ascending=True).reset_index(drop=True)
df.tail(11).reset_index(drop=True).style.background_gradient(cmap ='seismic').set_properties(**{'font-size': '11px'}).set_properties(**{'text-align': 'left'})

# Lexicon

In [ ]:
#lex[lex['RATING']!=0]
#lex[lex['Previous Rating']!=0]
# non-zero rating words
lex_nonzero = lex[lex['RATING']!=0]

#lex[lex['Previous Rating']>=3]
# rating words gerater than or equal to 3
#lex_nonzero = lex[lex['RATING']>=3]
lex_nonzero

## Hash through each comment to find only those that include non-zero lexicon words. 

In [ ]:
%%time
import time
from nltk.tokenize import word_tokenize

df_sample = df.sample(1000)

# hash through each comment to find only those that include non-zero lexicon words
lexicon_favored = df_sample.copy()
lexicon_favored['rating'] = pd.Series()
lexicon_favored['lexicon_word'] = pd.Series()
lexicon_favored['word_count'] = pd.Series()

for i,word in lex_nonzero.WORD.items():
    for i2,piece in df_sample.comments.items():     
        if word in word_tokenize(piece.lower()):
            #print('index',i2,'\nword',word, '\npiece',piece.lower(), '\nrating', lex_nonzero.RATING[i],'\n')

            # add rating from lexicon
            lexicon_favored['rating'][i2] = lex_nonzero.RATING[i]

            # add up every word usage in comments
            lexicon_favored['lexicon_word'][i2] = lex_nonzero.WORD[i]
            
            # word count
            lexicon_favored['word_count'][i2] = len(word_tokenize(piece.lower()))
        else:
            # word count
            lexicon_favored['word_count'][i2] = len(word_tokenize(piece.lower()))
            
lexicon_favored = lexicon_favored
#clear_output()

### Lexicon-Focused Dataset
<br>

#### Snippet

In [ ]:
lexicon_favored.head()

In [ ]:
any(lexicon_favored.index.duplicated())

In [ ]:
import matplotlib.pyplot as plt
_ = plt.hist(lexicon_favored.word_count, bins='auto')  # arguments are passed to np.histogram
plt.title("Word Counts Histogram with 'auto' bins")
plt.show()

In [ ]:
# stopwords
import nltk
from nltk.corpus import stopwords
 
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

lexicon_favored['just_words'] = pd.Series()
filtered_sentence = []

for i,piece in lexicon_favored.comments.items():     
    for word in word_tokenize(piece.lower()):
        if word not in stop_words:
            filtered_sentence.append({i, word})

### Ranked by Lexicon Rating.

In [ ]:
lexicon_favored = lexicon_favored.sort_values('rating',ascending=False).reset_index(drop=True)
lexicon_favored.head(25).style.background_gradient(cmap ='seismic').set_properties(**{'font-size': '11px'})

# "Be careful not to choke on your aspirations." - Darth Vader
### Application of [VADER](https://github.com/cjhutto/vaderSentiment) (Valence Aware Dictionary and sEntiment Reasoner)
A lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

In [ ]:
# https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/
# https://github.com/cjhutto/vaderSentiment
#
# Natural Language Toolkit: vader - TAMPERED w/ SCU Lexicon
#
# Copyright (C) 2001-2023 NLTK Project
# Author: C.J. Hutto <Clayton.Hutto@gtri.gatech.edu>
#         Ewan Klein <ewan@inf.ed.ac.uk> (modifications)
#         Pierpaolo Pantone <24alsecondo@gmail.com> (modifications)
#         George Berry <geb97@cornell.edu> (modifications)
#         Malavika Suresh <malavika.suresh0794@gmail.com> (modifications)
# URL: <https://www.nltk.org/>
"""
If you use the VADER sentiment analysis tools, please cite:

Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for
Sentiment Analysis of Social Media Text. Eighth International Conference on
Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
"""
import math
import re
import string
from itertools import product
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output
import nltk.data
clear_output()

class VaderConstants:
    """
    A class to keep the Vader lists and constants.
    """

    ##Constants##
    # (empirically derived mean sentiment intensity rating increase for booster words)
    B_INCR = 0.293
    B_DECR = -0.293

    # (empirically derived mean sentiment intensity rating increase for using
    # ALLCAPs to emphasize a word)
    C_INCR = 0.733

    N_SCALAR = -0.74

    NEGATE = {'UFO',
             'again',
             'alien',
             'another',
             'attack',
             'beam',
             'beautiful',
             'bed',
             'bedroom',
             'being',
             'believe',
             'capture',
             'caught',
             'chase',
             'chemtrail',
             'cloud',
             'clouds',
             'coming',
             'confuse',
             'contact',
             'daily',
             "didn't",
             'discover',
             'dream',
             'earth',
             'every',
             'excited',
             'experience',
             'figure',
             'fireball',
             'firework',
             'fireworks',
             'frequent',
             'happy',
             'help',
             'hope',
             'less',
             'me',
             'mufon',
             'needle',
             'nightly',
             'often',
             'orange',
             'pain',
             'paranormal',
             'please',
             'presence',
             'psychic',
             'review',
             'room',
             'scared',
             'seen',
             'shock',
             'shocked',
             'skin',
             'sleep',
             'star-like',
             'starlike',
             'starship',
             'storm',
             'surgery',
             'taken',
             'their',
             'thigh',
             'tragedy',
             'trance',
             'ufo',
             "ufo's",
             'ufos',
             'visit',
             'visitation',
             'want',
             'zoom',
             'zoomed'
             }


    # booster/dampener 'intensifiers' or 'degree adverbs'
    # https://en.wiktionary.org/wiki/Category:English_degree_adverbs

    BOOSTER_DICT = {
        'northeastern'  : B_INCR,
        'law'  : B_INCR,
        'duty'  : B_INCR,
        'appearing'  : B_INCR,
        'background'  : B_INCR,
        'clockwise'  : B_INCR,
        'momentarily'  : B_INCR,
        'degrees'  : B_INCR,
        'appearance'  : B_INCR,
        'approx'  : B_INCR,
        'approximate'  : B_INCR,
        'within'  : B_INCR,
        'NORAD'  : B_INCR,
        'plasma'  : B_INCR,
        'astronomical'  : B_INCR,
        'Lockheed'  : B_INCR,
        'retire'  : B_INCR,
        'locomotion'  : B_INCR,
        'feather'  : B_INCR,
        'seemed'  : B_INCR,
        'witnesses'  : B_INCR,
        'aviation'  : B_INCR,
        'nuclear'  : B_INCR,
        'air force'  : B_INCR,
        'north'  : B_INCR,
        'seem'  : B_INCR,
        'fort'  : B_INCR,
        'register'  : B_INCR,
        'translucent'  : B_INCR,
        'hexagon'  : B_INCR,
        'normal'  : B_INCR,
        'target'  : B_INCR,
        'morning'  : B_INCR,
        'rotar'  : B_INCR,
        'southwest'  : B_INCR,
        'AF'  : B_INCR,
        'south'  : B_INCR,
        'approach'  : B_INCR,
        'opaque'  : B_INCR,
        'hexagonal'  : B_INCR,
        'guard'  : B_INCR,
        'briefly'  : B_INCR,
        'rectangular'  : B_INCR,
        'upward'  : B_INCR,
        'officer'  : B_INCR,
        'telescope'  : B_INCR,
        'propulsion'  : B_INCR,
        'perimeter'  : B_INCR,
        'traverse'  : B_INCR,
        'army'  : B_INCR,
        'eastern'  : B_INCR,
        'elongate'  : B_INCR,
        'customer'  : B_INCR,
        'binoculars'  : B_INCR,
        'pulled'  : B_INCR,
        'emanate'  : B_INCR,
        'scope'  : B_INCR,
        'southern'  : B_INCR,
        'assume'  : B_INCR,
        'capability'  : B_INCR,
        'radar'  : B_INCR,
        'structure'  : B_INCR,
        'tree-line'  : B_INCR,
        'appear'  : B_INCR,
        'western'  : B_INCR,
        'defense'  : B_INCR,
        'irregular'  : B_INCR,
        'call'  : B_INCR,
        'analyze'  : B_INCR,
        'coast  '  : B_INCR,
        'vertex'  : B_INCR,
        'altitude'  : B_INCR,
        'grid'  : B_INCR,
        'blend'  : B_INCR,
        'public'  : B_INCR,
        'northern'  : B_INCR,
        'simultaneous'  : B_INCR,
        'perspective'  : B_INCR,
        'capable'  : B_INCR,
        'navy'  : B_INCR,
        'degree'  : B_INCR,
        'family'  : B_INCR,
        'east'  : B_INCR,
        'enforcement'  : B_INCR,
        'position'  : B_INCR,
        'F14'  : B_INCR,
        'astronomy'  : B_INCR,
        'police'  : B_INCR,
        'base'  : B_INCR,
        'security'  : B_INCR,
        'velocity'  : B_INCR,
        'similar'  : B_INCR,
        'civilian'  : B_INCR,
        'pilot'  : B_INCR,
        'F15'  : B_INCR,
        'southeastern'  : B_INCR,
        'Boeing'  : B_INCR,
        'counterclockwise'  : B_INCR,
        'estimate'  : B_INCR,
        'sentry'  : B_INCR,
        'sheriff'  : B_INCR,
        'assign'  : B_INCR,
        'B2'  : B_INCR,
        'cloak'  : B_INCR,
        'west'  : B_INCR,
        'northwest'  : B_INCR,
        'military'  : B_INCR,
        'marine'  : B_INCR,
        'engine'  : B_INCR,
        'reception'  : B_INCR,
        'treeline'  : B_INCR,
        '911'  : B_INCR,
        'radio'  : B_INCR,
        'southeast'  : B_INCR,
        'height'  : B_INCR,
        'rotating'  : B_INCR,
        'rotate'  : B_INCR,
        'octagonal'  : B_INCR,
        'day'  : B_INCR,
        'physics'  : B_INCR,
        'border'  : B_INCR,
        'northwestern'  : B_INCR,
        'cluster'  : B_INCR,
        'appeared'  : B_INCR,
        'solid'  : B_INCR,
        'rotary'  : B_INCR,
        'horizontal'  : B_INCR,
        'approximately'  : B_INCR,
        'equidistant'  : B_INCR,
        'independent'  : B_INCR,
        'naval'  : B_INCR,
        'policeman'  : B_INCR,
        'resemble'  : B_INCR,
        'northeast'  : B_INCR,
        'commercial'  : B_INCR,
        'southwestern'  : B_INCR,
        'octagon'  : B_INCR,
        'motion'  : B_INCR,
        'deputy'  : B_INCR,
        'daytime'  : B_INCR,
        'flight'  : B_INCR,
        'biologist'  : B_INCR,
        'silo'  : B_INCR,
        'patrol'  : B_INCR,
        'chemist'  : B_INCR,
        'perfect'  : B_INCR,
        'astronomer'  : B_INCR,
        'azimuth'  : B_INCR,
        'radioed'  : B_INCR,
        'elevation'  : B_INCR,
        'coast guard'  : B_INCR,
        'engineer'  : B_INCR,
        'reports'  : B_DECR,
        'towards'  : B_DECR,
        'directions'  : B_DECR,
        'attention'  : B_DECR,
        'directly'  : B_DECR,
        'friends'  : B_DECR,
        'triangle'  : B_DECR,
        'ft'  : B_DECR,
        'pictures'  : B_DECR,
        'event'  : B_DECR,
        'father'  : B_DECR,
        'observe '  : B_DECR,
        'observing'  : B_DECR,
        'extremely'  : B_DECR,
        'standing'  : B_DECR,
        'visible'  : B_DECR,
        'spherical'  : B_DECR,
        'angle'  : B_DECR,
        'hover'  : B_DECR,
        'triangular'  : B_DECR,
        'location'  : B_DECR,
        'diameter'  : B_DECR,
        'direction'  : B_DECR,
        'close'  : B_DECR,
        'speed'  : B_DECR,
        'hovered'  : B_DECR,
        'between'  : B_DECR,
        'object'  : B_DECR,
        'glow'  : B_DECR,
        'sister'  : B_DECR,
        'clearly'  : B_DECR,
        'never'  : B_DECR,
        'trees'  : B_DECR,
        'son'  : B_DECR,
        'near'  : B_DECR,
        'facing'  : B_DECR,
        'movement'  : B_DECR,
        'realized'  : B_DECR,
        'photos'  : B_DECR,
        'behind'  : B_DECR,
        'witnessed'  : B_DECR,
        'cloudy'  : B_DECR,
        'immediate'  : B_DECR,
        'notice'  : B_DECR,
        'camera'  : B_DECR,
        'large'  : B_DECR,
        'brother'  : B_DECR,
        'underneath'  : B_DECR,
        'asked'  : B_DECR,
        'closer'  : B_DECR,
        'called'  : B_DECR,
        'photo'  : B_DECR,
        'yards'  : B_DECR,
        'highway'  : B_DECR,
        'down'  : B_DECR,
        'objects'  : B_DECR,
        'video'  : B_DECR,
        'assumed'  : B_DECR,
        'horizon'  : B_DECR,
        'tree'  : B_DECR,
        'station'  : B_DECR,
        'mile'  : B_DECR,
        'shape'  : B_DECR,
        'together'  : B_DECR,
        'ground'  : B_DECR,
        'mountain'  : B_DECR,
        'mother'  : B_DECR,
        'observed'  : B_DECR,
        'curious'  : B_DECR,
        'fairly'  : B_DECR,
        'feet'  : B_DECR,
        'miles'  : B_DECR,
        'ocean'  : B_DECR,
        'below'  : B_DECR,
        'brightness'  : B_DECR,
        'bottom'  : B_DECR,
        'hill'  : B_DECR,
        'pattern'  : B_DECR,
        'remember'  : B_DECR,
        'daughter'  : B_DECR,
        'hovering'  : B_DECR,
        'joke'  : B_DECR,
        'stationary'  : B_DECR,
        'metallic'  : B_DECR,
        'probably'  : B_DECR,
        'top'  : B_DECR,
        'might'  : B_DECR,
        'approaching'  : B_DECR,
        'slightly'  : B_DECR,
        'smaller'  : B_DECR,
        'toward'  : B_DECR,
        'above'  : B_DECR,
        'different'  : B_DECR,
        'husband'  : B_DECR,
        'glowing'  : B_DECR,
        'mom'  : B_DECR,
        'water'  : B_DECR,
        'wasnt'  : B_DECR,
        'path'  : B_DECR,
        'friend'  : B_DECR,
        'dad'  : B_DECR,
        'observation'  : B_DECR,
        'center'  : B_DECR,
        'began'  : B_DECR,
        'course'  : B_DECR,
        'smoke'  : B_DECR,
        'cigar'  : B_DECR,
        'located'  : B_DECR,
        'overhead'  : B_DECR,
        'minutes'  : B_DECR,
        'dog'  : B_DECR,
        'witness'  : B_DECR,
        'cold'  : B_DECR,
        'report'  : B_DECR,
        'wife'  : B_DECR,
        'appears'  : B_DECR,
        'bright'  : B_DECR,
        'moon'  : B_DECR,
        'moving'  : B_DECR,
        'craft'  : B_DECR,
        'brighter'  : B_DECR,
    }

    # check for special case idioms using a sentiment-laden keyword known to SAGE
    SPECIAL_CASE_IDIOMS = {
        "the shit": 3,
        "the bomb": 3,
        "bad ass": 1.5,
        "yeah right": -2,
        "cut the mustard": 2,
        "kiss of death": -1.5,
        "hand to mouth": -2,
    }

    # for removing punctuation
    REGEX_REMOVE_PUNCTUATION = re.compile(f"[{re.escape(string.punctuation)}]")

    PUNC_LIST = [
        ".",
        "!",
        "?",
        "&",
        "#44",
        ",",
        ";",
        ":",
        "-",
        "'",
        '"',
        "!!",
        "!!!",
        "??",
        "???",
        "?!?",
        "!?!",
        "?!?!",
        "!?!?",
    ]

    def __init__(self):
        pass

    def pairwise(iterable):
        """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
    
    def negated(self, input_words, include_nt=True):
        """
        Determine if input contains negation words
        """
        neg_words = self.NEGATE
        if any(word.lower() in neg_words for word in input_words):
            return True
        if include_nt:
            if any("n't" in word.lower() for word in input_words):
                return True
        #for first, second in pairwise(input_words):
        #    if second.lower() == "least" and first.lower() != "at":
        #        return True
        return False


    def normalize(self, score, alpha=15):
        """
        Normalize the score to be between -1 and 1 using an alpha that
        approximates the max expected value
        """
        norm_score = score / math.sqrt((score * score) + alpha)
        return norm_score


    def scalar_inc_dec(self, word, valence, is_cap_diff):
        """
        Check if the preceding words increase, decrease, or negate/nullify the
        valence
        """
        scalar = 0.0
        word_lower = word.lower()
        if word_lower in self.BOOSTER_DICT:
            scalar = self.BOOSTER_DICT[word_lower]
            if valence < 0:
                scalar *= -1
            # check if booster/dampener word is in ALLCAPS (while others aren't)
            if word.isupper() and is_cap_diff:
                if valence > 0:
                    scalar += self.C_INCR
                else:
                    scalar -= self.C_INCR
        return scalar



class SentiText:
    """
    Identify sentiment-relevant string-level properties of input text.
    """

    def __init__(self, text, punc_list, regex_remove_punctuation):
        if not isinstance(text, str):
            text = str(text.encode("utf-8"))
        self.text = text
        self.PUNC_LIST = punc_list
        self.REGEX_REMOVE_PUNCTUATION = regex_remove_punctuation
        self.words_and_emoticons = self._words_and_emoticons()
        # doesn't separate words from
        # adjacent punctuation (keeps emoticons & contractions)
        self.is_cap_diff = self.allcap_differential(self.words_and_emoticons)


    def _words_plus_punc(self):
        """
        Returns mapping of form:
        {
            'cat,': 'cat',
            ',cat': 'cat',
        }
        """
        no_punc_text = self.REGEX_REMOVE_PUNCTUATION.sub("", self.text)
        # removes punctuation (but loses emoticons & contractions)
        words_only = no_punc_text.split()
        # remove singletons
        words_only = {w for w in words_only if len(w) > 1}
        # the product gives ('cat', ',') and (',', 'cat')
        punc_before = {"".join(p): p[1] for p in product(self.PUNC_LIST, words_only)}
        punc_after = {"".join(p): p[0] for p in product(words_only, self.PUNC_LIST)}
        words_punc_dict = punc_before
        words_punc_dict.update(punc_after)
        return words_punc_dict

    def _words_and_emoticons(self):
        """
        Removes leading and trailing puncutation
        Leaves contractions and most emoticons
            Does not preserve punc-plus-letter emoticons (e.g. :D)
        """
        wes = self.text.split()
        words_punc_dict = self._words_plus_punc()
        wes = [we for we in wes if len(we) > 1]
        for i, we in enumerate(wes):
            if we in words_punc_dict:
                wes[i] = words_punc_dict[we]
        return wes

    def allcap_differential(self, words):
        """
        Check whether just some words in the input are ALL CAPS

        :param list words: The words to inspect
        :returns: `True` if some but not all items in `words` are ALL CAPS
        """
        is_different = False
        allcap_words = 0
        for word in words:
            if word.isupper():
                allcap_words += 1
        cap_differential = len(words) - allcap_words
        if 0 < cap_differential < len(words):
            is_different = True
        return is_different



class SentimentIntensityAnalyzer:
    """
    Give a sentiment intensity score to sentences.
    """

    def __init__(
        self,
        lexicon_file="sentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt",
    ):
        self.lexicon_file = nltk.data.load(lexicon_file)
        self.lexicon = self.make_lex_dict()
        self.constants = VaderConstants()


    def make_lex_dict(self):
        """
        Convert lexicon file to a dictionary
        """
        lex_dict = {}
        for line in self.lexicon_file.split("\n"):
            (word, measure) = line.strip().split("\t")[0:2]
            lex_dict[word] = float(measure)
        return lex_dict


    def polarity_scores(self, text):
        """
        Return a float for sentiment strength based on the input text.
        Positive values are positive valence, negative value are negative
        valence.

        :note: Hashtags are not taken into consideration (e.g. #BAD is neutral). If you
            are interested in processing the text in the hashtags too, then we recommend
            preprocessing your data to remove the #, after which the hashtag text may be
            matched as if it was a normal word in the sentence.
        """
        # text, words_and_emoticons, is_cap_diff = self.preprocess(text)
        sentitext = SentiText(
            text, self.constants.PUNC_LIST, self.constants.REGEX_REMOVE_PUNCTUATION
        )
        sentiments = []
        words_and_emoticons = sentitext.words_and_emoticons
        for item in words_and_emoticons:
            valence = 0
            i = words_and_emoticons.index(item)
            if (
                i < len(words_and_emoticons) - 1
                and item.lower() == "kind"
                and words_and_emoticons[i + 1].lower() == "of"
            ) or item.lower() in self.constants.BOOSTER_DICT:
                sentiments.append(valence)
                continue

            sentiments = self.sentiment_valence(valence, sentitext, item, i, sentiments)

        sentiments = self._but_check(words_and_emoticons, sentiments)

        return self.score_valence(sentiments, text)


    def sentiment_valence(self, valence, sentitext, item, i, sentiments):
        is_cap_diff = sentitext.is_cap_diff
        words_and_emoticons = sentitext.words_and_emoticons
        item_lowercase = item.lower()
        if item_lowercase in self.lexicon:
            # get the sentiment valence
            valence = self.lexicon[item_lowercase]

            # check if sentiment laden word is in ALL CAPS (while others aren't)
            if item.isupper() and is_cap_diff:
                if valence > 0:
                    valence += self.constants.C_INCR
                else:
                    valence -= self.constants.C_INCR

            for start_i in range(0, 3):
                if (
                    i > start_i
                    and words_and_emoticons[i - (start_i + 1)].lower()
                    not in self.lexicon
                ):
                    # dampen the scalar modifier of preceding words and emoticons
                    # (excluding the ones that immediately preceed the item) based
                    # on their distance from the current item.
                    s = self.constants.scalar_inc_dec(
                        words_and_emoticons[i - (start_i + 1)], valence, is_cap_diff
                    )
                    if start_i == 1 and s != 0:
                        s = s * 0.95
                    if start_i == 2 and s != 0:
                        s = s * 0.9
                    valence = valence + s
                    valence = self._never_check(
                        valence, words_and_emoticons, start_i, i
                    )
                    if start_i == 2:
                        valence = self._idioms_check(valence, words_and_emoticons, i)

                        # future work: consider other sentiment-laden idioms
                        # other_idioms =
                        # {"back handed": -2, "blow smoke": -2, "blowing smoke": -2,
                        #  "upper hand": 1, "break a leg": 2,
                        #  "cooking with gas": 2, "in the black": 2, "in the red": -2,
                        #  "on the ball": 2,"under the weather": -2}

            valence = self._least_check(valence, words_and_emoticons, i)

        sentiments.append(valence)
        return sentiments


    def _least_check(self, valence, words_and_emoticons, i):
        # check for negation case using "least"
        if (
            i > 1
            and words_and_emoticons[i - 1].lower() not in self.lexicon
            and words_and_emoticons[i - 1].lower() == "least"
        ):
            if (
                words_and_emoticons[i - 2].lower() != "at"
                and words_and_emoticons[i - 2].lower() != "very"
            ):
                valence = valence * self.constants.N_SCALAR
        elif (
            i > 0
            and words_and_emoticons[i - 1].lower() not in self.lexicon
            and words_and_emoticons[i - 1].lower() == "least"
        ):
            valence = valence * self.constants.N_SCALAR
        return valence

    def _but_check(self, words_and_emoticons, sentiments):
        words_and_emoticons = [w_e.lower() for w_e in words_and_emoticons]
        but = {"but"} & set(words_and_emoticons)
        if but:
            bi = words_and_emoticons.index(next(iter(but)))
            for sidx, sentiment in enumerate(sentiments):
                if sidx < bi:
                    sentiments[sidx] = sentiment * 0.5
                elif sidx > bi:
                    sentiments[sidx] = sentiment * 1.5
        return sentiments

    def _idioms_check(self, valence, words_and_emoticons, i):
        onezero = f"{words_and_emoticons[i - 1]} {words_and_emoticons[i]}"

        twoonezero = "{} {} {}".format(
            words_and_emoticons[i - 2],
            words_and_emoticons[i - 1],
            words_and_emoticons[i],
        )

        twoone = f"{words_and_emoticons[i - 2]} {words_and_emoticons[i - 1]}"

        threetwoone = "{} {} {}".format(
            words_and_emoticons[i - 3],
            words_and_emoticons[i - 2],
            words_and_emoticons[i - 1],
        )

        threetwo = "{} {}".format(
            words_and_emoticons[i - 3], words_and_emoticons[i - 2]
        )

        sequences = [onezero, twoonezero, twoone, threetwoone, threetwo]

        for seq in sequences:
            if seq in self.constants.SPECIAL_CASE_IDIOMS:
                valence = self.constants.SPECIAL_CASE_IDIOMS[seq]
                break

        if len(words_and_emoticons) - 1 > i:
            zeroone = f"{words_and_emoticons[i]} {words_and_emoticons[i + 1]}"
            if zeroone in self.constants.SPECIAL_CASE_IDIOMS:
                valence = self.constants.SPECIAL_CASE_IDIOMS[zeroone]
        if len(words_and_emoticons) - 1 > i + 1:
            zeroonetwo = "{} {} {}".format(
                words_and_emoticons[i],
                words_and_emoticons[i + 1],
                words_and_emoticons[i + 2],
            )
            if zeroonetwo in self.constants.SPECIAL_CASE_IDIOMS:
                valence = self.constants.SPECIAL_CASE_IDIOMS[zeroonetwo]

        # check for booster/dampener bi-grams such as 'sort of' or 'kind of'
        if (
            threetwo in self.constants.BOOSTER_DICT
            or twoone in self.constants.BOOSTER_DICT
        ):
            valence = valence + self.constants.B_DECR
        return valence

    def _never_check(self, valence, words_and_emoticons, start_i, i):
        if start_i == 0:
            if self.constants.negated([words_and_emoticons[i - 1]]):
                valence = valence * self.constants.N_SCALAR
        if start_i == 1:
            if words_and_emoticons[i - 2] == "never" and (
                words_and_emoticons[i - 1] == "so"
                or words_and_emoticons[i - 1] == "this"
            ):
                valence = valence * 1.5
            elif self.constants.negated([words_and_emoticons[i - (start_i + 1)]]):
                valence = valence * self.constants.N_SCALAR
        if start_i == 2:
            if (
                words_and_emoticons[i - 3] == "never"
                and (
                    words_and_emoticons[i - 2] == "so"
                    or words_and_emoticons[i - 2] == "this"
                )
                or (
                    words_and_emoticons[i - 1] == "so"
                    or words_and_emoticons[i - 1] == "this"
                )
            ):
                valence = valence * 1.25
            elif self.constants.negated([words_and_emoticons[i - (start_i + 1)]]):
                valence = valence * self.constants.N_SCALAR
        return valence

    def _punctuation_emphasis(self, sum_s, text):
        # add emphasis from exclamation points and question marks
        ep_amplifier = self._amplify_ep(text)
        qm_amplifier = self._amplify_qm(text)
        punct_emph_amplifier = ep_amplifier + qm_amplifier
        return punct_emph_amplifier

    def _amplify_ep(self, text):
        # check for added emphasis resulting from exclamation points (up to 4 of them)
        ep_count = text.count("!")
        if ep_count > 4:
            ep_count = 4
        # (empirically derived mean sentiment intensity rating increase for
        # exclamation points)
        ep_amplifier = ep_count * 0.292
        return ep_amplifier

    def _amplify_qm(self, text):
        # check for added emphasis resulting from question marks (2 or 3+)
        qm_count = text.count("?")
        qm_amplifier = 0
        if qm_count > 1:
            if qm_count <= 3:
                # (empirically derived mean sentiment intensity rating increase for
                # question marks)
                qm_amplifier = qm_count * 0.18
            else:
                qm_amplifier = 0.96
        return qm_amplifier

    def _sift_sentiment_scores(self, sentiments):
        # want separate positive versus negative sentiment scores
        pos_sum = 0.0
        neg_sum = 0.0
        neu_count = 0
        for sentiment_score in sentiments:
            if sentiment_score > 0:
                pos_sum += (
                    float(sentiment_score) + 1
                )  # compensates for neutral words that are counted as 1
            if sentiment_score < 0:
                neg_sum += (
                    float(sentiment_score) - 1
                )  # when used with math.fabs(), compensates for neutrals
            if sentiment_score == 0:
                neu_count += 1
        return pos_sum, neg_sum, neu_count

    def score_valence(self, sentiments, text):
        if sentiments:
            sum_s = float(sum(sentiments))
            # compute and add emphasis from punctuation in text
            punct_emph_amplifier = self._punctuation_emphasis(sum_s, text)
            if sum_s > 0:
                sum_s += punct_emph_amplifier
            elif sum_s < 0:
                sum_s -= punct_emph_amplifier

            compound = self.constants.normalize(sum_s)
            # discriminate between positive, negative and neutral sentiment scores
            pos_sum, neg_sum, neu_count = self._sift_sentiment_scores(sentiments)

            if pos_sum > math.fabs(neg_sum):
                pos_sum += punct_emph_amplifier
            elif pos_sum < math.fabs(neg_sum):
                neg_sum -= punct_emph_amplifier

            total = pos_sum + math.fabs(neg_sum) + neu_count
            pos = math.fabs(pos_sum / total)
            neg = math.fabs(neg_sum / total)
            neu = math.fabs(neu_count / total)

        else:
            compound = 0.0
            pos = 0.0
            neg = 0.0
            neu = 0.0

        sentiment_dict = {
            "neg": round(neg, 3),
            "neu": round(neu, 3),
            "pos": round(pos, 3),
            "compound": round(compound, 4),
        }

        return sentiment_dict


# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    # create a list
    results = []
    results.append({"% Positive":sentiment_dict['pos'],
                    "% Negative":sentiment_dict['neg'],
                    "% Neutral":sentiment_dict['neu'],
                    "% Compound":sentiment_dict['compound']
                   })
    results = pd.DataFrame(results)
    return results

# Apply to df['comments'] column.
def NLP_PowellScore(commentsColumns):
    
    # obtain each comment for 'comments' column
    eachComment = [eachComment for i,eachComment in commentsColumns.items()]
    eachComment = pd.Series(eachComment)
                               
    # vader.variables.PowellScore
    PowellPositive = [v for v in list([sentiment_scores(sentimentAnalyzedComment)["% Positive"][0] for i,sentimentAnalyzedComment in eachComment.items()])]
    PowellNegative = [v for v in list([sentiment_scores(sentimentAnalyzedComment)["% Negative"][0] for i,sentimentAnalyzedComment in eachComment.items()])]
    PowellNeutral = [v for v in list([sentiment_scores(sentimentAnalyzedComment)["% Neutral"][0] for i,sentimentAnalyzedComment in eachComment.items()])]
    PowellCompound = [v for v in list([sentiment_scores(sentimentAnalyzedComment)["% Compound"][0] for i,sentimentAnalyzedComment in eachComment.items()])]
    
    return PowellPositive,PowellNegative,PowellNeutral,PowellCompound

### Reports sorted by Veracity
We also added the "Rating" score from Lexicon.

In [ ]:
# let's only take a small sample - this will definitely take a few minutes, grab yourself some water...
robert = lexicon_favored.copy()#.sample(30000)
robert.rating.value_counts()

In [ ]:
%%time
# defining Powell Scores by sentiment outputs: Positive, Negative, Neutral, Compound, & Rating
robert["PowellPositive"] = NLP_PowellScore(robert['comments'])[0]
robert["PowellNegative"] = NLP_PowellScore(robert['comments'])[1]
robert["PowellNeutral"] = NLP_PowellScore(robert['comments'])[2]
robert["PowellCompound"] = NLP_PowellScore(robert['comments'])[3]

# PowellScore 
robert["PowellScore"] = robert["PowellCompound"]#(robert["PowellPositive"]-robert["PowellNegative"])/robert["PowellNeutral"]

# veracity
robert["veracity"] = robert["PowellScore"]*robert["comment_length"]*robert["rating"] # FINALLY, THIS EQUATION ACCOUNTS FOR POWELL'S LEXICON RATINGS!

# veracity is still very much in progress. we are looking for ways forward to 
# better define them. although, it is important to note that "veracity" will 
# be variable that is subjective to the type of datasets.
columns = ['date posted','city','state','shape','comments',
           'comment_length','latitude','longitude','duration (minutes)',\
           'PowellScore','veracity','rating','lexicon_word']

df1 = robert[columns].sort_values('veracity',ascending=False).reset_index(drop=True)
df1[df1['comment_length']>10].head(20)\
        .style.background_gradient(cmap ='seismic').set_properties(**{'font-size': '11px'})

## Ovals seen in California

In [ ]:
%%time
# ca_oval
ca_oval = df1[df1.state=='ca'].reset_index(drop=True)
ca_oval = ca_oval[ca_oval['shape']=='oval']

# only ovals
ca_oval_162 = ca_oval.sort_values(['latitude','longitude'])
robert_ca_oval_162 = ca_oval_162
robert_ca_oval_162["PowellPositive"] = NLP_PowellScore(robert_ca_oval_162['comments'])[0]
robert_ca_oval_162["PowellNegative"] = NLP_PowellScore(robert_ca_oval_162['comments'])[1]
robert_ca_oval_162["PowellNeutral"] = NLP_PowellScore(robert_ca_oval_162['comments'])[2]

# PowellScore 
robert_ca_oval_162["PowellScore"] = (robert_ca_oval_162["PowellPositive"]-robert_ca_oval_162["PowellNegative"])/robert_ca_oval_162["PowellNeutral"]

# veracity
robert_ca_oval_162["veracity"] = robert_ca_oval_162["PowellScore"]*robert_ca_oval_162["comment_length"]*robert_ca_oval_162["rating"]


df2 = robert_ca_oval_162[columns].sort_values(['veracity'],ascending=False).reset_index(drop=True)
df2.head(50).style.background_gradient(cmap ='seismic').set_properties(**{'font-size': '11px'})

## Powell Variables in 3D
This is a 3D-interactive chart that uses the date posted, veracity, & PowellScore variables. Colored by lexicon rating. Sized by comment_length.

By definition, these actually render 5-Dimensional charts, if one considers veracity & commenth lengths of reports as 'features of a situation'.

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

df_plot = df1[df1.rating>0]
# graph
fig = px.scatter_3d(df_plot, x='date posted', y='veracity', z='PowellScore',
              color='rating',
              size = 'comment_length',
              hover_name = 'city',
              hover_data=['city','state','comments','rating','lexicon_word'],              
              opacity=0.5,
              size_max=17
                   )
fig.show()

### This one shows Date Posted vs PowellScore & Lexicon Rating Variables of California Oval reports. Colored by Veracity.

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# graph
fig = px.scatter_3d(df1, x='date posted', y='PowellScore', z='rating',
              color='veracity',
              size = 'comment_length',
              hover_name = 'city',
              hover_data=['city','state','comments','rating','lexicon_word'],              
              opacity=0.5,
              size_max=17
                   )
fig.show()

### Ovals seen in California - Date Posted vs PowellNeutral vs PowellScore

In [ ]:
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# graph
fig = px.scatter_3d(robert_ca_oval_162, x='date posted', y='PowellNeutral', z='rating',
              color='veracity',
              size = 'comment_length',
              hover_name = 'city',
              hover_data=['city','state','comments','shape','veracity','rating'],              
              opacity=0.5,
              size_max=17
                   )
fig.show()

These variables are still very much in progress & there currently is no process for defining them. Despite the disparate, disconnected, & wide range of skeptic/non-skeptic relational databases — we have managed to connect with organizations that promote open source — public repositories & most are willing to coordinate with one another in developing a UAP Reporting & Events Hub. Wherein all pertinent reports, sightings, measurements, & signatures are to be populated by various factors from multiple disciplines & technologies. We will do our best to coordinate with prominent key members of the UAP community in order to contribute to building out a “standardized” reporting mechanism in an intelligible & non-duplicative fashion. We are looking for ways forward in getting access to real-time, current reports.

The goal would be to create something similar to an Order of Battle, so that reports at specific times & locations can be compared to past reports to augment credibility determination, as well as eventually be compared to known events that may explain them. Once those explanations are vetted, reports would be coded by likelihood of mundane vs anomalous, which would aid in the processing of similar events in the future.

In addition, we have already begun looking for trends over time, such as the time of day when reported events take place, & the type of object reported over the decades. The latter can be observed in the “Shapes by Share of Reports” chart, which provides indications of confirmation bias in observed behavior.

Finally, big data analysis (alongside robust AI|ML|DS modeling techniques) could also provide insight into the development of improved collection & reporting processes, which currently appear to be undefined, improving the quality of the data we receive. — K. Kolbe.

# Different NLP Methods

In [ ]:
df1[['comments']].sample(15).style.set_properties(**{'text-align': 'left'})

In [ ]:
# lets split up every word from every column & frame that in itself, call it "words"
a = [v.split(' ') for i,v in df1.comments.items()]
flatlist=[]
for sublist in a:
    for element in sublist:
        flatlist.append(element)
comments = pd.DataFrame(flatlist, columns=['words'])
comments

In [ ]:
pd.set_option('display.max_rows', None)

comments.words.value_counts().head(20)

In [ ]:
comments.words.value_counts().head(50).plot(kind='barh',figsize=(15,7))

In [ ]:
# billy-boy!
# isolate only "adjectives, nouns, verbs, & adverbs"

# Visual Insights

## Word Clouds
### All 500 samples.

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# in the clouds
# 500 samples 
comment_words = ''
stopwords = set(STOPWORDS)
 
# iterate through the csv file
for val in df1.comments:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 1200, height = 600,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 7,
                colormap='twilight').generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize=(25,10), facecolor='None')
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

### California Ovals 

In [ ]:
# in the clouds
# california ovals
comment_words = ''
stopwords = set(STOPWORDS)
 
# iterate through the csv file
for val in df2.comments:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 1200, height = 600,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 7,
                colormap='twilight').generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize=(25,10), facecolor='None')
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

### Top Veracity Comments

In [ ]:
df1.reset_index(drop=True).set_index('veracity').head(20)[['comments','rating','lexicon_word','shape','city']]

### California Ovals Comments

In [ ]:
df2.set_index('veracity')[['comments','shape','city']]

# MUFON Data Exploitation
## DaScient-VADER Sentiment Analyzer

Here, we begin focusing on the lexicon analysis of each comment submission.

In [ ]:
%%time
import pandas as pd
!pip install xlrd
# Pull in data
mufon = pd.read_excel('/kaggle/input/scu-nlp-uap-lexicon/Powell with Comments.xls',sheet_name='mufon_cms_2017-04-09',header=0)
witnesses = pd.read_excel('/kaggle/input/scu-nlp-uap-lexicon/Powell with Comments.xls',sheet_name='Sheet1',header=0)
mufon = mufon.dropna(how='all').copy()
mufon.head()

In [ ]:
# diminish the mufon dataset
import numpy as np
mufon1 = mufon[mufon.Witnesses>0]
mufon1 = mufon1[mufon1.Length<3000]

In [ ]:
mufon1.head(3)

In [ ]:
%%time
import time
from nltk.tokenize import word_tokenize

df_sample1 = mufon1.sample(5000)

# hash through each comment to find only those that include non-zero lexicon words
lexicon_favored1 = df_sample1.copy()
lexicon_favored1['rating'] = pd.Series()
lexicon_favored1['lexicon_word'] = pd.Series()
lexicon_favored1['word_count'] = pd.Series()

for i,word in lex_nonzero.WORD.items():
    for i2,piece in df_sample1['Detailed Description'].items():
        if word in word_tokenize(piece.lower()):
            #print('index',i2,'\nword',word, '\npiece',piece.lower(), '\nrating', lex_nonzero.RATING[i],'\n')

            # add rating from lexicon
            lexicon_favored1['rating'][i2] = lex_nonzero.RATING[i]

            # add up every word usage in comments
            lexicon_favored1['lexicon_word'][i2] = lex_nonzero.WORD[i]
            
            # word count
            lexicon_favored1['word_count'][i2] = len(word_tokenize(piece.lower()))
        else:
            # word count
            lexicon_favored1['word_count'][i2] = len(word_tokenize(piece.lower()))
#clear_output()

In [ ]:
lexicon_favored1.to_csv('lexicon_favored1.csv',index=True)

In [ ]:
# Natural Language Toolkit: vader (VADER SCU UAP Lexicon - Notebook)
# One idea for diminishing computation time - Separate corpora by Date of Submission or by some known variable.
PowellScore = (0,1) = (Emotional, Scientific)
MODEL = 

In [ ]:
%%time
#mufon = mufon.sample(30)
# defining Powell Scores by sentiment outputs: Positive, Negative, & Neutral
lexicon_favored1["PowellPositive"] = NLP_PowellScore(lexicon_favored1['Detailed Description'])[0]
lexicon_favored1["PowellNegative"] = NLP_PowellScore(lexicon_favored1['Detailed Description'])[1]
lexicon_favored1["PowellNeutral"] = NLP_PowellScore(lexicon_favored1['Detailed Description'])[2]
lexicon_favored1["PowellCompound"] = NLP_PowellScore(lexicon_favored1['Detailed Description'])[3]

# PowellScore 
lexicon_favored1["PowellScore"] = (lexicon_favored1["PowellPositive"]-lexicon_favored1["PowellNegative"])/lexicon_favored1["PowellNeutral"]

# veracity
lexicon_favored1["veracity"] = lexicon_favored1["PowellScore"]*lexicon_favored1["Length"]*lexicon_favored1["Score"]

df3 = lexicon_favored1.sort_values('veracity',ascending=False).reset_index(drop=True)
df3.head(100)\
        .style.background_gradient(cmap ='seismic').set_properties(**{'font-size': '11px'})

In [ ]:
df3.to_csv('mufon_lexicon.csv',index=False)

# Data Science on MUFON Reports

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

# encoding
from sklearn.preprocessing import LabelEncoder

def encode(df):
    lb_make = LabelEncoder()
    columns = df.columns.values.tolist()
    df_encoded = df[columns].copy()

    # categorize/encode
    for i in columns:
        df_encoded[i] = lb_make.fit_transform(df[i])

    # encoded
    return df_encoded


# encoded variable re-mapping
def encoding_remap(df, df_encoded, target):
    
    X_test = X_y_sets(df, target)[0][0]
    
    remap = pd.merge(df_encoded.loc[df_encoded.index.isin(X_test.index.values)][target].reset_index(),
              df.loc[df.index.isin(X_test.index.values)][[target]].reset_index(),on=['index'])
    
    remap[target] = [str(remap[f'{target}_y'][i]) for i,v in remap[f'{target}_x'].items()]
    remap['index'] = np.array([str(remap[f'{target}_x'][i]) for i,v in remap[f'{target}_x'].items()]).astype(int)
    remap=remap[[target,'index']]
    remap = remap.set_index('index').drop_duplicates().sort_values('index')
    
    return remap


# pairplot
import seaborn as sns
def pairplot(df, target):
    return sns.pairplot(df.sample(int(len(df/10000))),hue=target)
    
    
# create X,y variables for ML
from sklearn.model_selection import train_test_split
def X_y_sets(df, target):
    X = df.dropna().drop(columns=[target]).copy()
    y = df.dropna()[target].ravel().copy()
    
    return train_test_split(X, y, test_size=0.33, random_state=42), X, y

In [ ]:
# is scaling necessary?
# construction of ML dataframes
target = 'Disposition'#''veracity','PowellScore','NLP_PowellScore'

# copy
a = lexicon_favored1[['Witnesses','Score','Length','Nearest City','State',\
        'Object Shape Primary','Exact Latitude','Exact Longitude','Disposition','veracity','PowellScore']].copy()

# for the sake of computationa efficiency
a = a.head(10000).reset_index(drop=True)

In [ ]:
a

In [ ]:
# find random sample & save index for defining an encoded use-case
from random import randrange
idx = randrange(len(a))

# print random configuration item
print("\nThis is a randomly chosen subject we will try to predict.")
b = pd.DataFrame(a.loc[idx]).T
print(f"\nTarget:'{target}' value is: ",b.reset_index()[target][0],"\n")

# store sol'n
solution = str(b.reset_index()[target][0])

# print data point
b
# if this cell fails, try it again from step 1 - you ran into a null variable (i'll fix that soon enough)

In [ ]:
# categorize/encode entire dataframe(a)
c = encode(a)
print("\nOriginal dataframe encoded into something we can run a classifier against.\n")
c.sample(10).reset_index(drop=True).style.background_gradient(cmap ='Pastel1').set_properties(**{'font-size': '10px'})

In [ ]:
# 'comments' & 'country' - out
sns.pairplot(c[['Witnesses','Score','Length','Nearest City','State','Object Shape Primary','Exact Latitude','Exact Longitude','Disposition','veracity','PowellScore']]\
             .sample(100).copy(),
             hue=f'{target}',
             kind="kde",
             corner=True,
             palette="Paired"
            )

In [ ]:
# print encoded item
use_case = pd.DataFrame(c.loc[idx]).T.drop(columns=[target]) 

#c

# print encoded item w/out target info
data = c.drop(columns=[target]) 

print("\nThis is what our encoded 'use-case' looks like - number form, just the way the machine likes it.\n")

use_case.style.background_gradient(cmap ='twilight').set_properties(**{'font-size': '10px'})

In [ ]:
# create X,y variables for ML
# save trainer
print("\nResetting train data...\nCreating X-matrix & y-vector (target) for classification.")
trainer = c.loc[c.index!=idx].copy()
X, y =  trainer.drop(columns=[target]), trainer[target].ravel()
X_train, X_test, y_train, y_test = X_y_sets(trainer, target)[0]

In [ ]:
X_train['target'] = pd.Series(y_train)
X_train.dropna().head().reset_index(drop=True).reset_index(drop=True).style.background_gradient(cmap ='twilight').set_properties(**{'font-size': '10px'})

In [ ]:
# for the sake of adding the 'target' column above for sake of layman's explanation
X_train, X_test, y_train, y_test = X_y_sets(trainer, target)[0]

In [ ]:
# encoded variable re-mapping
# specific to our current target choice
d = encoding_remap(a, c, target)
print("\nDecoding our encoded dataframe to correlate with the initial randomly chosen subject.\n")

In [ ]:
print("\n-Live prediction-\nThinking...\n")

# MLP
clf = MLPClassifier(alpha=0.666, max_iter=666).fit(X_train, y_train)

print()
print("Test score (confidence): ",clf.score(X_test, y_test)*100,"%")
print()
prediction = clf.predict(use_case)[0]
print(f"Prediction {target} index:",prediction)

# print decoded prediction
print("\nPrediction Decoded")
e = d[d.index == prediction]
e

In [ ]:
solved = str(e['Disposition'][e.index[0]])
if solution == solved:
    print(f"\nYUP!\n\nThe machine's prediction against target variable '{target}' was correct!\n")
else:
    print("\nNOPE!\nThe machine's prediction was incorrect :(")
    
print()

In [ ]:
# en fin